# ML for AE Term Project 

In this term project, we will use the results from the following investigating into "meta-materials"

[1] B. Deng, A. Zareei, X. Ding, J. C. Weaver, C. H. Rycroft, and K. Bertoldi, “Inverse Design of Mechanical Metamaterials with Target Nonlinear Response via a Neural Accelerated Evolution Strategy,” Advanced Materials, vol. 34, no. 41, Sep. 2022, doi: https://doi.org/10.1002/adma.202206238.

See https://www.youtube.com/watch?v=_TWHBZOqBiM&t=119s for a brief video explanation of this topic. 

*All images are taken from this paper.*

## Problem Statement 

"Metamaterials" are a way to design materials in specific geometries such that applying strain to these materials makes them behave in specific nonlinear ways. In the elastic deformation region, most materials behave linearly i.e. when we deform them by a specific amount, they exhibit a proportional amount of stress. This is described by Young's Modulus, $E$. However, with specific *geometries* of the material, we can design materials that behave nonlinearly within the elastic region: 

<img src="images/nonlinear_stress_strain.png" alt="alt text"  style="display:block; margin-left:auto; margin-right:auto; width:50%;">

Notice how as the applied strain, $\epsilon$, increases the nominal stress in the material (the force needed to produce $\epsilon$ divided by the original cross-sectional area of the material) begins linearly and then abruptly drops as the material snaps into a different equilibrium state.

The question drawn from the paper that will form the basis of our project is: 

**"How can we use machine learning to efficiently predict the nominal stress, $\sigma$, needed for a certain strain, $\epsilon$, from the geometry of the material?"** 

The first step to any ML problem is formally defining what features we will observe about our system and what outputs we seek to predict. 

## Feature-Selection

We first need to formalize the design of the materials into a set of parameters used to train a machine learning model. Specifically, Deng+ et al. observe that we can entirely define a regular, repeating mesh using eight independent points along a lattice (see below). The vertical and horizontal perturbations to each independent point define a set of 16 input features to our model.  

<img src="images/coordinate_features.png" alt="alt text"  style="display:block; margin-left:auto; margin-right:auto; width:50%;">

We will also use the desired strain, $\epsilon$, defined as the ratio between deformation and the free-length of the material. This will allow us to pose this problem as, "given this specific metamaterial geometry, what stress, $\sigma$, is needed to produce a strain of $\epsilon$?" 

## Feature/Output Summary
**Inputs:**
* $\theta \in \mathbb{R}^{16}$: The 16-dimensional vector of coordinate perturbations defining the geometry of the metamaterial's mesh. 
* $\epsilon \in \mathbb{R}$: The desired strain we wish to produce in the material. 

**Outputs:** the nominal stress, $\sigma$, required to produce this strain. 

In [43]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm

Probs not gonna be feasible to do it like this:

In [76]:
X_df = pd.read_csv("geometry_input.csv", header=None)
Y_df = pd.read_csv("stress_output.csv", header=None)
eps_min = 0.0 
eps_max = -0.1 

# Renaming the columns to match the diagram 
geometry_col_names = []
for i in range(8):
    geometry_col_names += ["u_" + str(i+1), "v_" + str(i+1)]

X_df.columns = geometry_col_names

# Adding epsilon column to new input dataframe 

new_X = []
new_Y = []

for i, row in tqdm(Y_df.iterrows(), total = len(Y_df)):
    for j in range(100):
        temp_df = X_df.loc[i,:].copy()
        temp_df['eps'] = j/100 * eps_max 
        new_X.append(temp_df)
        new_Y.append(Y_df.loc[i, j])

new_X = pd.DataFrame(new_X)
new_Y = pd.DataFrame(new_Y)

100%|██████████| 30000/30000 [10:38<00:00, 46.98it/s]  
